this is a developer notebook for testing out red cell detection. 

If you find this notebook somewhere, ignore it! There are suite3d demos labeled "Demo-....." that are the official demos.


In [20]:
%reload_ext autoreload
%autoreload 2

import os 
from datetime import date
from matplotlib import pyplot as plt
import numpy as np

# we need to set the current path to the directory
# containing the suite3d repository, this hack should
# do the trick
os.chdir(os.path.dirname(os.path.abspath("")))

from suite3d.job import Job
from suite3d import io
from suite3d import plot_utils as plot

In [ ]:
# update this to point to the demo data! 
tifs = io.get_tif_paths(r"D:\localData\suite3d\tinya_test_data_with_red")
for tif in tifs: print(tif)

In [22]:
# Set the mandatory parameters
params = {
    # volume rate
    'fs': io.get_vol_rate(tifs[0]),
    
    # planes to analyze. 0 is typically the flyback, so we exclude it here
    'planes' : np.array([0,1,2,3,4,5,6,7,8]), 
    # number of planes recorded by scanimage, including the flyback
    'n_ch_tif' : 9,
    
    # Decay time of the Ca indicator in seconds. 1.3 for GCaMP6s. This example is for GCamP8m
    'tau' : 1.3,
    'lbm' : False, 
    'num_colors' : 2, # how many color channels were recorded by scanimage
    'functional_color_channel' : 0, # which color channel is the functional one

    'process_structural_channel': True, # whether to process the structural channel
    'structural_color_channel': 1, # which color channel is the structural one
    'clear_registered_structural_data': False, # whether to clear the registered structural data after the mean volume is calculated

     # voxel size in z,y,x in microns
    'voxel_size_um' : (20, 1.5, 1.5),

    # number of files to use for the initial pass
    # usually, ~500 frames is a good rule of thumb
    # we will just use 200 here for speed
    'n_init_files' :  1,

    # 3D GPU registration - fast! 
    '3d_reg' : False,
    'gpu_reg' : True,
    'nonrigid' : True,
    
    # note : 3D CPU is not supported yet
    'subtract_crosstalk' : False, # turn off some lbm-only features
    'fuse_strips' : False, # turn off some lbm-only features    
}

In [ ]:
# Create the job
job = Job(r'D:\localData\suite3d\runs','test-redcell', tifs = tifs,
          params=params, create=True, overwrite=True, verbosity = 3)

In [ ]:
job.run_init_pass()

In [25]:
# If you have large tiffs, split the large tiffs into files of size 100 after registration
job.params['split_tif_size'] = 100

In [7]:
# OPTIONAL: load and take a look at the reference image
summary = job.load_summary()
ref_img = summary['ref_img_3d']

structural = job.load_summary(structural=True)
ref_img_struct = structural['ref_img_3d']

# # view 1 plane at a time
# plot.show_img(ref_img[2], figsize=(3,4))
# plot.show_img(ref_img_struct[2], figsize=(3,4))

# # interactive 3D viewer
# plot.VolumeViewer(ref_img)


In [ ]:
job.register()

In [9]:
import napari
import tifffile as tf

mov_green = job.get_registered_movie(key="registered_fused_data", filename_filter="fused", structural=False)[:, :200].compute()
mov_red = job.get_registered_movie(key="registered_fused_data", filename_filter="fused", structural=True)[:, :200].compute()

# Create the viewer and set human-readable axis labels
viewer = napari.Viewer()
viewer.dims.axis_labels = ['Plane', 'Time', 'Y', 'X']

# Add each movie as a separate layer with additive blending
viewer.add_image(
    mov_red,
    name='Red channel',
    colormap='red',
    blending='additive',
    visible=True
)
viewer.add_image(
    mov_green,
    name='Green channel',
    colormap='green',
    blending='additive',
    visible=True
)

napari.run()

In [ ]:
corr_map = job.calculate_corr_map()

In [30]:
res = job.load_corr_map_results()
vmap = res['vmap']

In [ ]:
job.params['patch_size_xy'] = (550, 550)
# for speed, only segment a single patch
job.segment_rois()

In [ ]:
job.compute_npil_masks()
traces = job.extract_and_deconvolve()

In [ ]:
results_path = r'D:\localData\suite3d\results\test-redcell'
job.export_results(results_path, result_dir_name='rois')

To take a look at the outputs in napari, navigate to the suite3d directory in a command shell and run the following:
```
python curation.py curation --output_dir /path/to/output/rois
```

In [ ]:
# Code for developing red cell analysis for 3D ROIs

from pathlib import Path
import numpy as np
results_path = Path(r"D:\localData\suite3d\results\test-redcell\s3d-results-test-redcell")

stats = np.load(results_path / "stats.npy", allow_pickle=True)
iscell = np.load(results_path / "iscell.npy")

In [ ]:
stats[0]["lam"]

In [86]:
idxcells = np.where(iscell[:, 0])[0]
s = stats[idxcells[0]]